#### Importaciones

In [20]:
import requests
from pprint import pprint
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize
import datetime

#### Conexión con la base de datos

In [21]:
# Client connects to "localhost" by default
client = MongoClient("localhost:27017")
db = client.dataton2019

serverStatusResult = db.command("serverStatus")
## pprint(serverStatusResult)

#### Query a la base de datos

In [22]:
query ={}
projection = {'_id':0, 'ocid':1, 'parties.roles':1, 'parties.id':1, 'awards.value.amount':1, 'awards.suppliers.id':1, 'cycle':1, 'awards.id':1, 'tender.tenderPeriod.startDate':1}
res = db.contrataciones.find(query, projection)
resList = [item for item in res]

#### Definición de funciones auxiliares para crear DataFrame final

In [4]:
def assignRoles(item):
    """asignar roles: b, s, p, t"""
    string = ''
    if 'buyer' in item['roles']: string += 'b'
    if 'supplier' in item['roles']: string += 's'
    if 'procuringEntity' in item['roles']: string += 'p'
    if 'tenderer' in item['roles']: string += 't'
    return string


def createDicts(element, totList):
    """"""
    for item in element['parties']:
        d = {}
        d['ocid'] = element['ocid']
        d['id'] = item['id']
        d['roles'] = assignRoles(item)
        d['year'] = element['tender']['tenderPeriod']['startDate'].split('T')[0].split('-')[0]
        d['month'] = element['tender']['tenderPeriod']['startDate'].split('T')[0].split('-')[1]
        try:
            if (('b' in d['roles']) or ('p' in d['roles'])):
                d['awds'] =  sum([award['value']['amount'] for award in element['awards']])
            else: 
                d['awds'] = sum([award['value']['amount'] for award in element['awards'] if d['id']==award['suppliers'][0]['id']])
        except:
            d['awds'] = 0
        totList.append(d)

[{'ocid': 'ocds-07smqs-1003803',
  'id': 'SAT-284',
  'roles': 'b',
  'year': '2016',
  'month': '02',
  'awds': 8451072},
 {'ocid': 'ocds-07smqs-1003803',
  'id': 'SAT970701NN3-006E00002',
  'roles': 'p',
  'year': '2016',
  'month': '02',
  'awds': 8451072},
 {'ocid': 'ocds-07smqs-1003803',
  'id': 'E9C1C827AE1234CCF7AC4D9070BB597C',
  'roles': 'st',
  'year': '2016',
  'month': '02',
  'awds': 8451072}]

#### Creación de dataframe final

In [23]:
totList = [ ]
for item in resList:
    createDicts(item, totList)

contrat_df = pd.DataFrame(totList, columns=['id', 'ocid', 'roles', 'awds', 'month', 'year'])

# Eliminar parties duplicadas para un contrato dado
contrat_df.drop_duplicates(['id', 'ocid'], inplace=True)

# Exportación de datos
contrat_df.to_csv('contratacionesDF.csv', index=False)

#### Crear un diccionario con los IDs de las parties y sus nombres

In [3]:
query ={}
projection = {'_id':0, 'parties':1}
resul = [item for item in db.contrataciones.find(query, projection)]

entities_dict = {item2['id']: item2['name'] for item in resul for item2 in item['parties']}
entities_dict

keys = []
values=[]
for k, v in entities_dict.items():
    keys.append(k)
    values.append(v)
entitiesDF = pd.DataFrame({'id':keys, 'name': values})
entitiesDF.to_csv('entitiesDF.csv',index=False)

#### Crear un DataFrame con solamente los contratos más altos

In [24]:
data = pd.read_csv('contratacionesDF.csv')

listIDs = []
year = 2016
for year in [2016, 2017, 2018]:
    listIDs.extend(list(data.loc[(data.year==year) & (data.roles== 'p'), :].sort_values(by=['awds'], ascending=False)[:5].ocid))


short_data = data.loc[data.ocid.isin(listIDs), :]
short_data.to_csv('contratacionesShort.csv', index=False)
short_data